In [17]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd

In [18]:
# load data

df=pd.read_csv('Cardiotocography_02_v10.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,LB,AC,FM,UC,ASTV,MSTV,ALTV,MLTV,DL,DS,...,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,outlier
0,146.0,0.0,0.0,5.0,65.0,0.4,33.0,7.4,0.0,0.0,...,134.0,164.0,1.0,0.0,150.0,149.0,151.0,1.0,0.0,1
1,128.0,0.0,0.0,2.0,86.0,0.3,79.0,2.9,0.0,0.0,...,114.0,130.0,0.0,0.0,128.0,126.0,129.0,0.0,1.0,1
2,149.0,0.0,0.0,5.0,61.0,0.4,34.0,5.6,0.0,0.0,...,148.0,160.0,1.0,0.0,154.0,153.0,155.0,0.0,0.0,1
3,122.0,0.0,0.0,0.0,83.0,0.5,6.0,15.6,0.0,0.0,...,62.0,130.0,0.0,0.0,122.0,122.0,123.0,3.0,1.0,1
4,134.0,0.0,4.0,0.0,79.0,0.2,42.0,5.5,0.0,0.0,...,128.0,145.0,2.0,0.0,135.0,135.0,136.0,1.0,0.0,1


In [19]:
df.shape

(1688, 22)

In [20]:
# data to values
df = df.values

In [21]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Cardiotocography.json")
gc = GCForest(config)

In [22]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,21]
X = df[:,0:21]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [23]:
# number of class
len(np.unique(y_train))

2

In [24]:
         # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-05-15 10:17:12,998][cascade_classifier.fit_transform] X_groups_train.shape=[(1130, 21)],y_train.shape=(1130,),X_groups_test.shape=[(558, 21)],y_test.shape=(558,)
[ 2018-05-15 10:17:13,002][cascade_classifier.fit_transform] group_dims=[21]
[ 2018-05-15 10:17:13,006][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 10:17:13,009][cascade_classifier.fit_transform] group_ends=[21]
[ 2018-05-15 10:17:13,011][cascade_classifier.fit_transform] X_train.shape=(1130, 21),X_test.shape=(558, 21)
[ 2018-05-15 10:17:13,013][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(1130, 21), X_cur_test.shape=(558, 21)
[ 2018-05-15 10:17:13,516][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.37%
[ 2018-05-15 10:17:14,123][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.37%
[ 2018-05-15 10:17:14,744][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_

In [25]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-05-15 10:17:19,042][cascade_classifier.transform] X_groups_test.shape=[(558, 21)]
[ 2018-05-15 10:17:19,044][cascade_classifier.transform] group_dims=[21]
[ 2018-05-15 10:17:19,045][cascade_classifier.transform] X_test.shape=(558, 21)
[ 2018-05-15 10:17:19,046][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(558, 21)


Test Accuracy of ExtraTrees = 98.566308 %


In [26]:
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[549   2]
 [  6   1]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99       551
        1.0       0.33      0.14      0.20         7

avg / total       0.98      0.99      0.98       558



In [27]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-05-15 10:17:20,490][cascade_classifier.transform] X_groups_test.shape=[(558, 21)]
[ 2018-05-15 10:17:20,493][cascade_classifier.transform] group_dims=[21]
[ 2018-05-15 10:17:20,495][cascade_classifier.transform] X_test.shape=(558, 21)
[ 2018-05-15 10:17:20,497][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(558, 21)


In [28]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-05-15 10:17:21,711][cascade_classifier.fit_transform] X_groups_train.shape=[(1130, 23)],y_train.shape=(1130,),X_groups_test.shape=[(558, 23)],y_test.shape=(558,)
[ 2018-05-15 10:17:21,714][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-05-15 10:17:21,715][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 10:17:21,717][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-05-15 10:17:21,718][cascade_classifier.fit_transform] X_train.shape=(1130, 23),X_test.shape=(558, 23)
[ 2018-05-15 10:17:21,721][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(1130, 23), X_cur_test.shape=(558, 23)
[ 2018-05-15 10:17:22,210][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-05-15 10:17:22,790][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.37%
[ 2018-05-15 10:17:23,383][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_

In [29]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-05-15 10:17:27,679][cascade_classifier.transform] X_groups_test.shape=[(558, 23)]
[ 2018-05-15 10:17:27,683][cascade_classifier.transform] group_dims=[23]
[ 2018-05-15 10:17:27,685][cascade_classifier.transform] X_test.shape=(558, 23)
[ 2018-05-15 10:17:27,686][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(558, 23)


Test Accuracy of ExtraTrees = 99.103943 %


In [30]:
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[550   1]
 [  4   3]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00       551
        1.0       0.75      0.43      0.55         7

avg / total       0.99      0.99      0.99       558



In [31]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')
    # Matrix de confusion
    from sklearn.metrics import classification_report, confusion_matrix
    print(confusion_matrix(y_test,y_pred))
    print('\n')
    print(classification_report(y_test,y_pred))

In [32]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-05-15 10:17:29,200][cascade_classifier.fit_transform] X_groups_train.shape=[(1130, 23)],y_train.shape=(1130,),X_groups_test.shape=[(558, 23)],y_test.shape=(558,)
[ 2018-05-15 10:17:29,204][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-05-15 10:17:29,206][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 10:17:29,207][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-05-15 10:17:29,208][cascade_classifier.fit_transform] X_train.shape=(1130, 23),X_test.shape=(558, 23)
[ 2018-05-15 10:17:29,210][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(1130, 23), X_cur_test.shape=(558, 23)
[ 2018-05-15 10:17:29,699][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-05-15 10:17:30,291][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=95.61%
[ 2018-05-15 10:17:30,886][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_

('Test Accuracy of ExtraTrees = 98.924731 %', 0, 'fois')
[[549   2]
 [  4   3]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99       551
        1.0       0.60      0.43      0.50         7

avg / total       0.99      0.99      0.99       558



[ 2018-05-15 10:17:36,877][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-05-15 10:17:37,459][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=95.61%
[ 2018-05-15 10:17:38,043][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=94.74%
[ 2018-05-15 10:17:38,617][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.25%
[ 2018-05-15 10:17:39,198][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.23%
[ 2018-05-15 10:17:39,777][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.35%
[ 2018-05-15 10:17:40,366][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.11%
[ 2018-05-15 10:17:40,963][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.21%


('Test Accuracy of ExtraTrees = 99.283154 %', 1, 'fois')
[[551   0]
 [  4   3]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00       551
        1.0       1.00      0.43      0.60         7

avg / total       0.99      0.99      0.99       558



[ 2018-05-15 10:17:44,007][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-05-15 10:17:44,597][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=94.74%
[ 2018-05-15 10:17:45,199][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=94.74%
[ 2018-05-15 10:17:45,774][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.25%
[ 2018-05-15 10:17:46,373][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.23%
[ 2018-05-15 10:17:46,976][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.35%
[ 2018-05-15 10:17:47,573][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.11%
[ 2018-05-15 10:17:48,157][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.32%


...

('Test Accuracy of ExtraTrees = 98.924731 %', 17, 'fois')
[[551   0]
 [  6   1]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99       551
        1.0       1.00      0.14      0.25         7

avg / total       0.99      0.99      0.99       558



[ 2018-05-15 10:19:40,081][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.37%
[ 2018-05-15 10:19:40,668][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=92.98%
[ 2018-05-15 10:19:41,243][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.25%
[ 2018-05-15 10:19:41,848][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.37%
[ 2018-05-15 10:19:42,432][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.23%
[ 2018-05-15 10:19:43,018][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=96.46%
[ 2018-05-15 10:19:43,601][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.21%
[ 2018-05-15 10:19:44,208][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.21%


('Test Accuracy of ExtraTrees = 98.924731 %', 18, 'fois')
[[551   0]
 [  6   1]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99       551
        1.0       1.00      0.14      0.25         7

avg / total       0.99      0.99      0.99       558



[ 2018-05-15 10:19:47,272][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.37%
[ 2018-05-15 10:19:47,867][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=92.98%
[ 2018-05-15 10:19:48,461][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.37%
[ 2018-05-15 10:19:49,063][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.37%
[ 2018-05-15 10:19:49,734][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.23%
[ 2018-05-15 10:19:50,328][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.35%
[ 2018-05-15 10:19:50,903][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.21%
[ 2018-05-15 10:19:51,499][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.21%


('Test Accuracy of ExtraTrees = 98.566308 %', 19, 'fois')
[[549   2]
 [  6   1]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99       551
        1.0       0.33      0.14      0.20         7

avg / total       0.98      0.99      0.98       558



# ('Test Accuracy of ExtraTrees = 99.283154 %', 1, 'fois')